Add SD-128/198/260 file to your google drive.
http://xiaopingwu.cn/assets/projects/sd-198/

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cd drive/My\ Drive && cd sd-128 && cp SD-128.tar /content/SD-128.tar
!cd /content/ && tar -xvf SD-128.tar

images/Neurofibroma/022041HB.jpg
images/Dermatofibroma/000215VB.jpg
images/Pustular_Psoriasis/043429HB.jpg
images/Exfoliative_Erythroderma/040359HB.jpg
images/Callus/029424HB.jpg
images/Steroid_Acne/009013HB.jpg
images/Radiodermatitis/
images/Congenital_Nevus/011859HB.jpg
images/Dysplastic_Nevus/021711HB.jpg
images/Congenital_Nevus/009108VB.jpg
images/Keratosis_Pilaris/028877VB.jpg
images/Keratoacanthoma/018702VB.jpg
images/Skin_Tag/023976VB.jpg
images/Nevus_Incipiens/010425HB.jpg
images/Pitted_Keratolysis/029020VB.jpg
images/Angioma/042597HB.jpg
images/Ichthyosis/030716VB.jpg
images/Halo_Nevus/011124HB.jpg
images/Congenital_Nevus/010634HB.jpg
images/Keratosis_Pilaris/028870VB.jpg
images/Follicular_Mucinosis/009546HB.jpg
images/Nummular_Eczema/040663VB.jpg
images/Lymphocytic_Infiltrate_of_Jessner/015299VB.jpg
images/Nevus_Incipiens/005942HB.jpg
images/Seborrheic_Keratosis/045797HB.jpg
images/Acne_Keloidalis_Nuchae/009514HB1.jpg
images/Halo_Nevus/011136HB.jpg
images/Livedo_Reticularis/0

Remove non-relevant diseases such as those for the hair and nails.

In [0]:
!rm -rf /content/images/{Alopecia_Areata,Androgenetic_Alopecia,Scalp_Psoriasis,Clubbing_of_Fingers,Median_Nail_Dystrophy,Nail_Dystrophy,Nail_Psoriasis,Onychlosis,Onychomycosis,Onychoschizia}

In [0]:
!pip install fastai albumentations opencv-python matplotlib numpy

     |████████████████████████████████| 634kB 3.5MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654020 sha256=7aedd970d45c34120ad209bcb39d90f2cc6cb59586c13b8ce8de72bd3e1b1218
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9


In [0]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import shutil
from albumentations import (Compose, VerticalFlip, HorizontalFlip, Flip, RandomRotate90, Rotate, ShiftScaleRotate, CenterCrop, OpticalDistortion, GridDistortion, ElasticTransform, JpegCompression, HueSaturationValue,
                            RGBShift, RandomBrightness, RandomContrast, Blur, MotionBlur, MedianBlur, GaussNoise, CLAHE, ChannelShuffle, InvertImg, RandomGamma, ToGray, PadIfNeeded, OneOf, RandomBrightnessContrast
                           )

from os import listdir
from os.path import isfile, join
import os

print("Finish import")

import random

abs_file_dir = "/content/images/"

# https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory
folders = [f for f in listdir(abs_file_dir) if ((not (isfile(join(abs_file_dir, f)))))]
folders.sort()
print("Finished creating folder names")
print(folders)

folders = folders[110:]
print(folders)


Finish import
Finished creating folder names
['Acne_Keloidalis_Nuchae', 'Acne_Vulgaris', 'Actinic_solar_Damage(Actinic_Keratosis)', 'Actinic_solar_Damage(Cutis_Rhomboidalis_Nuchae)', 'Actinic_solar_Damage(Pigmentation)', 'Actinic_solar_Damage(Solar_Elastosis)', 'Actinic_solar_Damage(Solar_Purpura)', 'Actinic_solar_Damage(Telangiectasia)', 'Allergic_Contact_Dermatitis', 'Angioma', 'Angular_Cheilitis', 'Apocrine_Hydrocystoma', 'Basal_Cell_Carcinoma', "Beau's_Lines", 'Benign_Keratosis', 'Blue_Nevus', "Bowen's_Disease", 'Callus', 'Candidiasis', 'Cellulitis', 'Compound_Nevus', 'Congenital_Nevus', "Crowe's_Sign", 'Cutaneous_Horn', 'Darier-White_Disease', 'Dermatofibroma', 'Dermatosis_Papulosa_Nigra', 'Desquamation', 'Digital_Fibroma', 'Dilated_Pore_of_Winer', 'Disseminated_Actinic_Porokeratosis', 'Drug_Eruption', 'Dry_Skin_Eczema', 'Dyshidrosiform_Eczema', 'Dysplastic_Nevus', 'Eczema', 'Epidermal_Nevus', 'Epidermoid_Cyst', 'Erythema_Craquele', 'Erythema_Multiforme', 'Exfoliative_Erythroderma

In [0]:
# List of augmentations to be used
augmentation = Compose([
        HorizontalFlip(p=0.7),
        RandomGamma(gamma_limit=(90, 110), p=0.7),
        RandomBrightness(limit=0.21, p=0.7),
        RandomContrast(limit=0.2, p=0.7),
        JpegCompression(quality_lower=80, quality_upper=100, p=0.7),
        OneOf([
            CLAHE(p=0.75),
            RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.75),
            RGBShift(r_shift_limit=40,g_shift_limit=15,b_shift_limit=2, p=0.75),
            HueSaturationValue(hue_shift_limit=8, sat_shift_limit=12,
                        val_shift_limit=11, p=.75)
        ], p=0.75),
        OneOf([
            Rotate(p=0.25),
            ElasticTransform(alpha=80, sigma=80, alpha_affine=80, p=0.5),
        ], p=0.75)
    ], p=1) 


print("Formed options")

for folder in folders:
    disease_dir = abs_file_dir + folder + "/"
    images = [i for i in listdir(disease_dir) if (isfile(join(disease_dir, i)))]
    for image in images:
        image_file_path = disease_dir + image
        image = cv2.imread(image_file_path)
        print("current disease filepath: " + image_file_path)
        for number in range(13):
            aug = augmentation(image=image)
            augmented_image = aug['image']
            print(augmentation)
            augmented_image_filepath = image_file_path.split(".")[0] + "_" + str(random.randint(1, 10000)) + ".jpg"
            cv2.imwrite(augmented_image_filepath, augmented_image)
            print("saved image filepath: " + augmented_image_filepath)

    shutil.move(disease_dir, "/content/drive/My Drive/aug_images")

Formed options
current disease filepath: /content/images/Tinea_Corporis/037579HB.jpg
saved image filepath: /content/images/Tinea_Corporis/037579HB_2013.jpg
saved image filepath: /content/images/Tinea_Corporis/037579HB_241.jpg
saved image filepath: /content/images/Tinea_Corporis/037579HB_609.jpg
saved image filepath: /content/images/Tinea_Corporis/037579HB_1134.jpg
saved image filepath: /content/images/Tinea_Corporis/037579HB_9407.jpg
saved image filepath: /content/images/Tinea_Corporis/037579HB_3314.jpg
saved image filepath: /content/images/Tinea_Corporis/037579HB_2899.jpg
saved image filepath: /content/images/Tinea_Corporis/037579HB_4379.jpg
saved image filepath: /content/images/Tinea_Corporis/037579HB_8329.jpg
saved image filepath: /content/images/Tinea_Corporis/037579HB_1402.jpg
saved image filepath: /content/images/Tinea_Corporis/037579HB_3906.jpg
saved image filepath: /content/images/Tinea_Corporis/037579HB_5738.jpg
saved image filepath: /content/images/Tinea_Corporis/037579HB_829